In [21]:
import pandas as pd
import uuid
import re
from openpyxl import load_workbook
from openpyxl.styles import Font
import os

def correct_csv(file_path, expected_fields):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    corrected_lines = []
    for line in lines:
        fields = line.strip().split(',')
        if len(fields) > expected_fields:
            corrected_fields = fields[:expected_fields]
        else:
            corrected_fields = fields
        corrected_lines.append(','.join(corrected_fields) + '\n')

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(corrected_lines)

def trasponi_valore_accanto_header1(file_path, expected_fields,UPLOAD_FOLDER):
    try:
        # Assicurati che il file CSV sia nel formato corretto
        correct_csv(file_path, expected_fields)

        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        values_next_to_header1 = []
        data_rows = []
        header_pattern = re.compile(r'^HEADER\d+')
        value_next_to_header1 = ""

        for line in lines:
            fields = line.strip().split(',')
            if fields[0] == "HEADER1":
                value_next_to_header1 = fields[1] if len(fields) > 1 else ""
            if not header_pattern.match(fields[0]):
                data_rows.append(fields)
                values_next_to_header1.append(value_next_to_header1)

        df_data = pd.DataFrame(data_rows)

        # Gestione della colonna SKU
        if len(df_data.columns) >= 3:
            df_data['SKU'] = df_data.iloc[:, 1] + '-' + df_data.iloc[:, 2]

        df_data['Value_Next_to_HEADER1'] = values_next_to_header1
        df_data = df_data[['Value_Next_to_HEADER1', 'SKU'] + [col for col in df_data.columns if col not in ['SKU', 'Value_Next_to_HEADER1']]]

        # Genera un nome di file unico per evitare sovrascritture
        unique_filename = f"output_{uuid.uuid4()}.xlsx"
        temp_output_file_path = os.path.join(UPLOAD_FOLDER, unique_filename)

        headers = ["Rif. Sped.", "SKU", "Collo", "Codice", "Colore", "Size", "UPC", "Made in", "Unità", "Confezione", "customer PO", "Riferimento Spedizione"]
        df_data.columns = headers[:len(df_data.columns)]

        df_data[['Collo', 'UPC']] = df_data[['Collo', 'UPC']].apply(lambda x: '{:.0f}'.format(x) if isinstance(x, (int, float)) else x)

        df_data.to_excel(temp_output_file_path, index=False, float_format="%.0f")

        # Applica lo stile alle celle dell'intestazione
        wb = load_workbook(temp_output_file_path)
        ws = wb.active
        for col_num in range(1, len(headers) + 1):
            cell = ws.cell(row=1, column=col_num)
            cell.font = Font(bold=True)
        wb.save(temp_output_file_path)

        return temp_output_file_path

    except Exception as e:
        print(f"Si è verificato un errore: {e}")
        return None

In [22]:
path = "uploads/Nike CSV-18e1a8c7be0.CSV"
UPLOAD_FOLDER = "downloads"

trasponi_valore_accanto_header1(path,9,UPLOAD_FOLDER)

'downloads\\output_dbe2745f-f5a1-4c40-815f-60a11dbe66bc.xlsx'